In [ ]:
%load_ext autoreload
%autoreload 2

from nnTrain import train_efn, train_pfn, get_data, get_data_softDrop
import matplotlib.pyplot as plt
import numpy as np
from multiprocessing import Process
from tensorflow import keras
from keras.optimizers import Adam
from tensorflow import keras
from keras import backend as K
from energyflow.utils import data_split, to_categorical
from sklearn.model_selection import train_test_split
import gc
from tensorflow.keras.losses import BinaryCrossentropy

# define rng seeds for train/test/val split for replicable results
rng1 = 45446
rng2 = 25135

# Quark jets, R = 0.8

In [ ]:
base="/uscms/home/sbrightt/nobackup/jets-ml/datasets/safeIncalculable_v2/"
nmax = 200000

hdata = get_data('q',hlevel=True,efrac=True,base=base,nmax=nmax,wta=False)
htrain, htest = train_test_split(hdata,train_size=0.5,random_state=rng1)
htest, hval = train_test_split(htest,train_size=0.5,random_state=rng2)

pdata = get_data('q',hlevel=False,efrac=True,base=base,nmax=nmax,wta=False)
ptrain, ptest = train_test_split(pdata,train_size=0.5,random_state=rng1)
ptest, pval = train_test_split(ptest,train_size=0.5,random_state=rng2)

train = np.concatenate((htrain,ptrain),axis=0)
train_labels = np.concatenate((np.ones((htrain.shape[0],1)),np.zeros((ptrain.shape[0],1))),axis=0)
perm = np.random.permutation(train.shape[0])
train = train[perm]
train_labels = train_labels[perm]

test = np.concatenate((htest,ptest),axis=0)
test_labels = np.concatenate((np.ones((htest.shape[0],1)),np.zeros((ptest.shape[0],1))),axis=0)
perm = np.random.permutation(test.shape[0])
test = test[perm]
test_labels = test_labels[perm]

val = np.concatenate((hval,pval),axis=0)
val_labels = np.concatenate((np.ones((hval.shape[0],1)),np.zeros((pval.shape[0],1))),axis=0)
perm = np.random.permutation(val.shape[0])
val = val[perm]
val_labels = val_labels[perm]

del hdata, pdata, htrain, htest, hval, ptrain, ptest, pval, perm

In [ ]:
# Regular EFN, no spectral normalization
loss = BinaryCrossentropy(from_logits=False)
#out_act = keras.activations.linear
model_kwargs = {"Phi_sizes":(60,60,60), "F_sizes":(80,80,80),
                "F_dropouts":0.1,
                "input_dim":2,
                "output_dim":1,"output_act":"sigmoid",
                "patience":5,
                "optimizer":Adam(1e-3),
                "loss":loss,
                "summary":True}
model_kwargs['spectral'] = False
train_kwargs = {"epochs":100,"verbose":1,"batch_size":512}
efn, auc, efn_fp, efn_tp, threshs = train_efn((train,train_labels),
                                              (test,test_labels),
                                              (val,val_labels),
                                              model_kwargs,
                                              train_kwargs,
                                              plot=True)
efn.save("keras_models_v2/qjet_EFN_zpT_iter1.keras")
del efn
gc.collect()

In [ ]:
# EFN with spectral normalization
for i in range(6,11):
    loss = BinaryCrossentropy(from_logits=False)
    #out_act = keras.activations.linear
    model_kwargs = {"Phi_sizes":(60,60,60), "F_sizes":(80,80,80),
                    "F_dropouts":0.1,
                    "input_dim":2,
                    "output_dim":1,"output_act":"sigmoid",
                    "patience":5,
                    "optimizer":Adam(1e-4),
                    "loss":loss,
                    "summary":True}
    model_kwargs['spectral'] = True
    train_kwargs = {"epochs":150,"verbose":1,"batch_size":10000}
    efn, auc, efn_fp, efn_tp, threshs = train_efn((train,train_labels),
                                                  (test,test_labels),
                                                  (val,val_labels),
                                                  model_kwargs,
                                                  train_kwargs,
                                                  plot=True)
    efn.save(f"keras_models_v2/qjet_spectralEFN_zpT_iter{i}.keras")
    del efn
    gc.collect()

## L = 1 study

In [ ]:
rtrain = np.concatenate([train[:,:,:1],np.linalg.norm(train[:,:,1:],axis=-1,keepdims=True)],axis=-1)
rtest = np.concatenate([test[:,:,:1],np.linalg.norm(test[:,:,1:],axis=-1,keepdims=True)],axis=-1)
rval = np.concatenate([val[:,:,:1],np.linalg.norm(val[:,:,1:],axis=-1,keepdims=True)],axis=-1)

In [ ]:
# Regular EFN, no spectral normalization
loss = BinaryCrossentropy(from_logits=False)
L = 2
model_kwargs = {"Phi_sizes":(150,150,L), "F_sizes":(100,100,100),
                "F_dropouts":0.1,
                "input_dim":1,
                "output_dim":1,"output_act":"sigmoid",
                "patience":5,
                "optimizer":Adam(1e-4),
                "loss":loss,
                "summary":True}
model_kwargs['spectral'] = False
train_kwargs = {"epochs":100,"verbose":1,"batch_size":128}
efn, auc, efn_fp, efn_tp, threshs = train_efn((rtrain,train_labels),
                                              (rtest,test_labels),
                                              (rval,val_labels),
                                              model_kwargs,
                                              train_kwargs,
                                              plot=True)
#efn.save(f"keras_models_Lsmall/qjet_EFN_zpT_L1.keras")
#del efn
gc.collect()

In [ ]:
import tensorflow.keras.backend as K
model = efn.model
kf = K.function(model.layers[0].input,model.layers[8].output)
def eval_filter(kf,R,n=100):
    xmin, ymin, xmax, ymax = -R, -R, R, R
    nx = ny = n
    # construct grid of inputs
    xs, ys = np.linspace(xmin, xmax, nx), np.linspace(ymin, ymax, ny)
    X, Y = np.meshgrid(xs, ys, indexing='ij')
    XY = np.asarray([X, Y]).reshape((1, 2, nx*ny)).transpose((0, 2, 1))
    # evaluate function
    Z = kf(XY)[0].reshape(nx, ny, L).transpose((2, 0, 1))
    return X, Y, Z

In [ ]:
import tensorflow.keras.backend as K
model = efn.model
kf = K.function(model.layers[0].input,model.layers[8].output)
R = 0.1
rads = np.linspace(0,R,1000).reshape(-1,1)
outs = kf(rads)

In [ ]:
plt.plot(rads[:,0],outs[:,0,:])

In [ ]:
from sklearn.metrics import roc_auc_score
betas = np.linspace(0,20,100)
from tqdm import tqdm
aucs = []
for b in tqdm(betas):
    ang = np.sum(rtest[:,:,0]*np.power(rtest[:,:,1],b,out=np.zeros_like(rtest[:,:,1]),where=rtest[:,:,1]!=0),axis=1)
    aucs.append(roc_auc_score(test_labels[:,0],ang))
plt.plot(betas,aucs)

In [ ]:
from matplotlib.colors import LogNorm, Normalize
from scipy.optimize import curve_fit
R = 1.0
#X,Y,Z = efn.eval_filters(R,prune=False,n=1000)
X,Y,Z = eval_filter(kf,R,n=1000)
colors = ["Reds","Blues","Greens","Oranges","Purples","Greys","YlOrBr","RdPu"]
fig,axes = plt.subplots(1,1,figsize=(5*len(Z),4))
for i in range(len(Z)):
    plt.sca(axes)
    norm = Normalize()
    plt.imshow(Z[i].T,origin='lower',extent=[-R,R,-R,R],aspect='auto',norm=norm)
    plt.colorbar()
    plt.xticks(rotation=45)

In [ ]:
test_model = keras.models.load_model("keras_models_Lsmall/qjet_EFN_zpT_L1.keras")
nev = test.shape[0]
splits = np.array_split(np.arange(nev),nev//10000)
preds = []
for s in splits:
    preds.append(test_model([test[s][:,:,0],test[s][:,:,1:]])[:,0].numpy())
preds = np.concatenate(preds)

In [ ]:
import sklearn.metrics as metrics
auc = metrics.roc_auc_score(test_labels[:,0],preds)
fpr,tpr,threshs = metrics.roc_curve(test_labels[:,0],preds)
plt.plot(tpr,1-fpr)
print(auc)

## L = 1 spectral

In [ ]:
# Regular EFN, no spectral normalization
loss = BinaryCrossentropy(from_logits=False)
L = 1
model_kwargs = {"Phi_sizes":(150,150,L), "F_sizes":(100,100,100),
                "F_dropouts":0.1,
                "input_dim":2,
                "output_dim":1,"output_act":"sigmoid",
                "patience":10,
                "optimizer":Adam(1e-4),
                "loss":loss,
                "summary":True}
model_kwargs['spectral'] = True
train_kwargs = {"epochs":200,"verbose":1,"batch_size":10000}
efn, auc, efn_fp, efn_tp, threshs = train_efn((train,train_labels),
                                              (test,test_labels),
                                              (val,val_labels),
                                              model_kwargs,
                                              train_kwargs,
                                              plot=True)
efn.save(f"keras_models_Lsmall/qjet_spectralEFN_zpT_L1.keras")
#del efn
gc.collect()

In [ ]:
import tensorflow.keras.backend as K
model = efn.model
kf = K.function(model.layers[0].input,model.layers[8].output)
def eval_filter(kf,R,n=100):
    xmin, ymin, xmax, ymax = -R, -R, R, R
    nx = ny = n
    # construct grid of inputs
    xs, ys = np.linspace(xmin, xmax, nx), np.linspace(ymin, ymax, ny)
    X, Y = np.meshgrid(xs, ys, indexing='ij')
    XY = np.asarray([X, Y]).reshape((1, 2, nx*ny)).transpose((0, 2, 1))
    # evaluate function
    Z = kf(XY)[0].reshape(nx, ny, L).transpose((2, 0, 1))
    return X, Y, Z

In [ ]:
from matplotlib.colors import LogNorm, Normalize
from scipy.optimize import curve_fit
R = 0.1
#X,Y,Z = efn.eval_filters(R,prune=False,n=1000)
X,Y,Z = eval_filter(kf,R,n=1000)
colors = ["Reds","Blues","Greens","Oranges","Purples","Greys","YlOrBr","RdPu"]
fig,axes = plt.subplots(1,1,figsize=(5*len(Z),4))
for i in range(len(Z)):
    plt.sca(axes)
    norm = Normalize()
    plt.imshow(Z[i].T,origin='lower',extent=[-R,R,-R,R],aspect='auto',norm=norm)
    plt.colorbar()
    plt.xticks(rotation=45)

# Gluon jets, R = 0.8

In [ ]:
base="/uscms/home/sbrightt/nobackup/jets-ml/datasets/safeIncalculable_v2/"
nmax = 200000

hdata = get_data('g',hlevel=True,efrac=False,base=base,nmax=nmax,wta=False)
htrain, htest = train_test_split(hdata,train_size=0.5,random_state=rng1)
htest, hval = train_test_split(htest,train_size=0.5,random_state=rng2)

pdata = get_data('g',hlevel=False,efrac=False,base=base,nmax=nmax,wta=False)
ptrain, ptest = train_test_split(pdata,train_size=0.5,random_state=rng1)
ptest, pval = train_test_split(ptest,train_size=0.5,random_state=rng2)

train = np.concatenate((htrain,ptrain),axis=0)
train_labels = np.concatenate((np.ones((htrain.shape[0],1)),np.zeros((ptrain.shape[0],1))),axis=0)
perm = np.random.permutation(train.shape[0])
train = train[perm]
train_labels = train_labels[perm]

test = np.concatenate((htest,ptest),axis=0)
test_labels = np.concatenate((np.ones((htest.shape[0],1)),np.zeros((ptest.shape[0],1))),axis=0)
perm = np.random.permutation(test.shape[0])
test = test[perm]
test_labels = test_labels[perm]

val = np.concatenate((hval,pval),axis=0)
val_labels = np.concatenate((np.ones((hval.shape[0],1)),np.zeros((pval.shape[0],1))),axis=0)
perm = np.random.permutation(val.shape[0])
val = val[perm]
val_labels = val_labels[perm]

del hdata, pdata, htrain, htest, hval, ptrain, ptest, pval, perm

In [ ]:
# Regular EFN, no spectral normalization
loss = BinaryCrossentropy(from_logits=False)
model_kwargs = {"Phi_sizes":(60,60,60), "F_sizes":(80,80,80),
                "F_dropouts":0.1,
                "input_dim":2,
                "output_dim":1,"output_act":"sigmoid",
                "patience":5,
                "optimizer":Adam(1e-3),
                "loss":loss,
                "summary":True}
model_kwargs['spectral'] = False
train_kwargs = {"epochs":100,"verbose":1,"batch_size":128}
efn, auc, efn_fp, efn_tp, threshs = train_efn((train,train_labels),
                                              (test,test_labels),
                                              (val,val_labels),
                                              model_kwargs,
                                              train_kwargs,
                                              plot=True)
efn_name = "keras_models_v1/gjet_EFN_zpT_iter10.keras"
efn.save(efn_name)
del efn
gc.collect()

In [ ]:
# EFN with spectral normalization
for i in range(6,11):
    loss = BinaryCrossentropy(from_logits=False)
    model_kwargs = {"Phi_sizes":(60,60,60), "F_sizes":(80,80,80),
                    "F_dropouts":0.1,
                    "input_dim":2,
                    "output_dim":1,"output_act":"sigmoid",
                    "patience":5,
                    "optimizer":Adam(1e-4),
                    "loss":loss,
                    "summary":True}
    model_kwargs['spectral'] = True
    train_kwargs = {"epochs":150,"verbose":1,"batch_size":10000}
    efn, auc, efn_fp, efn_tp, threshs = train_efn((train,train_labels),
                                                  (test,test_labels),
                                                  (val,val_labels),
                                                  model_kwargs,
                                                  train_kwargs,
                                                  plot=True)
    efn.save(f"keras_models_v2/gjet_spectralEFN_zpT_iter{i}.keras")
    del efn
    gc.collect()

## L = 1 study

In [ ]:
# Regular EFN, no spectral normalization
loss = BinaryCrossentropy(from_logits=False)
L = 2
model_kwargs = {"Phi_sizes":(100,100,L), "F_sizes":(100,100,100),
                "F_dropouts":0.1,
                "input_dim":2,
                "output_dim":1,"output_act":"sigmoid",
                "patience":10,
                "optimizer":Adam(1e-4),
                "loss":loss,
                "summary":True}
model_kwargs['spectral'] = False
train_kwargs = {"epochs":1000,"verbose":1,"batch_size":10000}
efn, auc, efn_fp, efn_tp, threshs = train_efn((train,train_labels),
                                              (test,test_labels),
                                              (val,val_labels),
                                              model_kwargs,
                                              train_kwargs,
                                              plot=True)
#efn.save(f"keras_models_Lsmall/gjet_EFN_zpT_L1.keras")
#del efn
gc.collect()

In [ ]:
import tensorflow.keras.backend as K
model = efn.model
kf = K.function(model.layers[0].input,model.layers[8].output)
def eval_filter(kf,R,n=100):
    xmin, ymin, xmax, ymax = -R, -R, R, R
    nx = ny = n
    # construct grid of inputs
    xs, ys = np.linspace(xmin, xmax, nx), np.linspace(ymin, ymax, ny)
    X, Y = np.meshgrid(xs, ys, indexing='ij')
    XY = np.asarray([X, Y]).reshape((1, 2, nx*ny)).transpose((0, 2, 1))
    # evaluate function
    Z = kf(XY)[0].reshape(nx, ny, L).transpose((2, 0, 1))
    return X, Y, Z

In [ ]:
# from matplotlib.colors import LogNorm, Normalize
from scipy.optimize import curve_fit
R = 0.5
#X,Y,Z = efn.eval_filters(R,prune=False,n=1000)
X,Y,Z = eval_filter(kf,R,n=1000)
colors = ["Reds","Blues","Greens","Oranges","Purples","Greys","YlOrBr","RdPu"]
fig,axes = plt.subplots(1,len(Z),figsize=(5*len(Z),4))
for i in range(len(Z)):
    plt.sca(axes[i])
    norm = Normalize()
    plt.imshow(Z[i].T,origin='lower',extent=[-R,R,-R,R],aspect='auto',norm=norm)
    plt.colorbar()
    plt.xticks(rotation=45)